<a href="https://colab.research.google.com/github/RogerioPiazzon/trabalhofinalPECE/blob/main/FineTuning-BERTimbau.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix,f1_score,classification_report
import numpy as np
import re,unicodedata,nltk
from nltk.corpus import stopwords
from transformers import AutoTokenizer, TFBertForSequenceClassification, BertConfig
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import keras
# import transformers
# import tensorflow_hub as hub
# from tqdm import tqdm
# import pickle
# from keras.models import Model
# import keras.backend as K
# from sklearn.metrics import confusion_matrix,f1_score,classification_report
# import matplotlib.pyplot as plt
# from keras.callbacks import ModelCheckpoint
# import itertools
# from keras.models import load_model
# from transformers import TrainingArguments, Trainer

nltk.download('stopwords')
stops_nltk = nltk.corpus.stopwords.words('portuguese')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
gpus = tf.config.list_physical_devices("GPU")
if gpus:
    for gpu in gpus:
        print("Found a GPU with the name:", gpu)
else:
    print("Failed to detect a GPU.")

Found a GPU with the name: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [ ]:
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def clean_stopwords_shortwords(w):
    stopwords_list=stopwords.words('portuguese')
    words = w.split()
    clean_words = [word for word in words if (word not in stopwords_list) and len(word) > 2]
    return " ".join(clean_words)

def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())
    # w = re.sub(r"([?.!,¿])", r" ", w)
    w = re.sub(r'[" "]+', " ", w)
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    # w=clean_stopwords_shortwords(w)
    w=re.sub(r'@\w+', '',w)
    return w

In [ ]:
df = pd.read_excel("/content/drive/MyDrive/PECE/resoucers/querys.xlsx",sheet_name='Planilha1')
df_to_use = df.loc[:,["Question","NEW INTENT"]].dropna()
data=df_to_use.rename(columns = {'NEW INTENT': 'label', 'Question': 'text'}, inplace = False)

In [ ]:
data=data.dropna()                                                           # Drop NaN valuues, if any
data=data.reset_index(drop=True)                                             # Reset index after dropping the columns/rows with NaN values
data = shuffle(data)                                                         # Shuffle the dataset
print('Available labels: ',data.label.unique())                              # Print all the unique labels in the dataset
# data['text']=data['text'].map(preprocess_sentence)
data['gt'] = pd.factorize(data['label'], sort=True)[0]

Available labels:  ['INFORMACAO_SOBRE_CLIENTE' 'LOCALIZAR_FUNCIONARIO'
 'MOVIMENTACAO_QUADRO_DE_FUNCIONARIOS' 'DEPARTAMENTO_DO_FUNCIONARIO'
 'LOCALIZAR_CLIENTE' 'CARGO_DO_FUNCIONARIO' 'METRICAS_DOS_CLIENTES'
 'METRICAS_DE_PRODUCAO_DE_PRODUTOS' 'METRICAS_DO_QUADRO_DE_FUNCIONARIOS'
 'METRICAS_DE_CUSTO_DE_PRODUTOS' 'LOCALIZAR_PRODUTOS_PELA_PRODUCAO'
 'LOCALIZAR_PRODUTOS_PELO_CUSTO' 'QUANTIDADE_DE_DEPARTAMENTOS'
 'METRICAS_DE_PRODUTOS_POR_CATEGORIA' 'TEMPO_DE_CASA_FUNCIONARIO']


In [ ]:
sentences=data['text']
labels=data['gt']
num_classes=len(data.label.unique())
len(sentences),len(labels)

(182, 182)

In [ ]:
num_classes

15

In [ ]:
model_id = 'neuralmind/bert-base-portuguese-cased'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = TFBertForSequenceClassification.from_pretrained(model_id,num_labels=num_classes,hidden_dropout_prob=0.5,attention_probs_dropout_prob=0.3)
#mexer no dropout pelo menos 60%

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier', 'bert/pooler/dense/bias:0', 'bert/pooler/dense/kernel:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.config

BertConfig {
  "_name_or_path": "neuralmind/bert-base-portuguese-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.3,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.5,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_10": 10,
    "LABEL_11": 11,
    "LABEL_12": 12,
    "LABEL_13": 13,
    "LABEL_14": 14,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7,
    "LABEL_8": 8,
    "LABEL_9": 9
  },
  "layer_norm_eps"

In [ ]:
input_ids=[]
attention_masks=[]

for sent in sentences:
    bert_inp=tokenizer.encode_plus(sent,add_special_tokens = True,max_length = 64, padding='max_length',return_attention_mask = True)
    input_ids.append(bert_inp['input_ids'])
    attention_masks.append(bert_inp['attention_mask'])

input_ids=np.asarray(input_ids)
attention_masks=np.array(attention_masks)
labels=np.array(labels)

In [ ]:
train_inp,val_inp,train_label,val_label,train_mask,val_mask=train_test_split(input_ids,labels,attention_masks,test_size=0.3)

In [ ]:
log_dir='/content/drive/MyDrive/PECE/tensorboard_data/tb_bertimbau'
model_save_path='/content/drive/MyDrive/PECE/models/bertimbau_model.h5'

callbacks = [
             keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_loss',mode='min',save_best_only=True),
             keras.callbacks.TensorBoard(log_dir=log_dir),
             keras.callbacks.EarlyStopping(monitor="val_loss",min_delta=1,patience=10,verbose=0,mode="min",start_from_epoch=30, baseline=0.1)
             ]

print('\nBERTimbau Model',model.summary())

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5,epsilon=1e-08)

model.compile(loss=loss,optimizer=optimizer,metrics=[metric])

Model: "tf_bert_for_sequence_classification_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  108923136 
                                                                 
 dropout_607 (Dropout)       multiple                  0         
                                                                 
 classifier (Dense)          multiple                  11535     
                                                                 
Total params: 108934671 (415.55 MB)
Trainable params: 108934671 (415.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________

BERTimbau Model None


In [ ]:
history=model.fit([train_inp,train_mask],train_label,batch_size=1,epochs=60,validation_data=([val_inp,val_mask],val_label),callbacks=callbacks)

Epoch 1/60
127/127 [==============================] - 10s 73ms/step - loss: 0.5948 - accuracy: 0.8346 - val_loss: 0.7110 - val_accuracy: 0.7455
Epoch 2/60
127/127 [==============================] - 6s 45ms/step - loss: 0.6676 - accuracy: 0.8268 - val_loss: 0.8348 - val_accuracy: 0.6909
Epoch 3/60
127/127 [==============================] - 6s 46ms/step - loss: 0.5793 - accuracy: 0.8268 - val_loss: 0.7592 - val_accuracy: 0.7091
Epoch 4/60
127/127 [==============================] - 6s 48ms/step - loss: 0.5242 - accuracy: 0.8504 - val_loss: 0.7808 - val_accuracy: 0.7636
Epoch 5/60
127/127 [==============================] - 9s 70ms/step - loss: 0.5045 - accuracy: 0.8819 - val_loss: 0.5882 - val_accuracy: 0.7636
Epoch 6/60
127/127 [==============================] - 6s 45ms/step - loss: 0.5245 - accuracy: 0.8740 - val_loss: 0.7169 - val_accuracy: 0.7636
Epoch 7/60
127/127 [==============================] - 6s 46ms/step - loss: 0.6245 - accuracy: 0.8268 - val_loss: 0.6732 - val_accuracy: 0.745

In [ ]:



import json

# Convert and write JSON object to file
with open("/content/drive/MyDrive/PECE/resoucers/dict_classes.json", "w") as outfile:
    json.dump(dict_classes, outfile)



In [ ]:
dict_classes = dict([i for i in zip(data['gt'].astype(str).values,data['label'].values)])
labels_dict = [dict_classes[str(i)] for i in [*range(len(list(dict_classes.keys())))]]

In [ ]:
model_id = 'neuralmind/bert-base-portuguese-cased'
model_save_path='/content/drive/MyDrive/PECE/models/bertimbau_model.h5'

tokenizer = AutoTokenizer.from_pretrained(model_id)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5,epsilon=1e-08)

trained_model = TFBertForSequenceClassification.from_pretrained(model_id,num_labels=num_classes)
# trained_model.compile(loss=loss,optimizer=optimizer, metrics=[metric])
trained_model.load_weights(model_save_path)

preds = trained_model.predict([val_inp,val_mask],batch_size=1)
pred_labels = np.argmax(preds.logits, axis=1)
f1 = f1_score(val_label,pred_labels,average='micro')
print('F1 score',f1)
print('Classification Report')
print(classification_report(val_label,pred_labels,target_names=labels_dict))

print('Training and saving built model.....')

#FOCO NO RECALL > 80%
#APOS ISSO PRECISAO

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier', 'bert/pooler/dense/bias:0', 'bert/pooler/dense/kernel:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


55/55 [==============================] - 4s 8ms/step
F1 score 0.9090909090909091
Classification Report
                                     precision    recall  f1-score   support

               CARGO_DO_FUNCIONARIO       1.00      1.00      1.00         5
        DEPARTAMENTO_DO_FUNCIONARIO       1.00      1.00      1.00         4
           INFORMACAO_SOBRE_CLIENTE       1.00      0.75      0.86         4
                  LOCALIZAR_CLIENTE       0.67      1.00      0.80         2
              LOCALIZAR_FUNCIONARIO       1.00      1.00      1.00         4
   LOCALIZAR_PRODUTOS_PELA_PRODUCAO       0.75      1.00      0.86         3
      LOCALIZAR_PRODUTOS_PELO_CUSTO       0.86      1.00      0.92         6
      METRICAS_DE_CUSTO_DE_PRODUTOS       0.50      1.00      0.67         1
   METRICAS_DE_PRODUCAO_DE_PRODUTOS       1.00      1.00      1.00         5
 METRICAS_DE_PRODUTOS_POR_CATEGORIA       1.00      0.25      0.40         4
              METRICAS_DOS_CLIENTES       0.86   

In [ ]:
dict_classes

{10: 'METRICAS_DOS_CLIENTES',
 0: 'CARGO_DO_FUNCIONARIO',
 12: 'MOVIMENTACAO_QUADRO_DE_FUNCIONARIOS',
 9: 'METRICAS_DE_PRODUTOS_POR_CATEGORIA',
 11: 'METRICAS_DO_QUADRO_DE_FUNCIONARIOS',
 6: 'LOCALIZAR_PRODUTOS_PELO_CUSTO',
 3: 'LOCALIZAR_CLIENTE',
 4: 'LOCALIZAR_FUNCIONARIO',
 7: 'METRICAS_DE_CUSTO_DE_PRODUTOS',
 2: 'INFORMACAO_SOBRE_CLIENTE',
 1: 'DEPARTAMENTO_DO_FUNCIONARIO',
 8: 'METRICAS_DE_PRODUCAO_DE_PRODUTOS',
 5: 'LOCALIZAR_PRODUTOS_PELA_PRODUCAO',
 13: 'QUANTIDADE_DE_DEPARTAMENTOS',
 14: 'TEMPO_DE_CASA_FUNCIONARIO'}

In [ ]:
dict([i for i in zip(data['gt'].values,data['label'].values)])

{10: 'METRICAS_DOS_CLIENTES',
 0: 'CARGO_DO_FUNCIONARIO',
 12: 'MOVIMENTACAO_QUADRO_DE_FUNCIONARIOS',
 9: 'METRICAS_DE_PRODUTOS_POR_CATEGORIA',
 11: 'METRICAS_DO_QUADRO_DE_FUNCIONARIOS',
 6: 'LOCALIZAR_PRODUTOS_PELO_CUSTO',
 3: 'LOCALIZAR_CLIENTE',
 4: 'LOCALIZAR_FUNCIONARIO',
 7: 'METRICAS_DE_CUSTO_DE_PRODUTOS',
 2: 'INFORMACAO_SOBRE_CLIENTE',
 1: 'DEPARTAMENTO_DO_FUNCIONARIO',
 8: 'METRICAS_DE_PRODUCAO_DE_PRODUTOS',
 5: 'LOCALIZAR_PRODUTOS_PELA_PRODUCAO',
 13: 'QUANTIDADE_DE_DEPARTAMENTOS',
 14: 'TEMPO_DE_CASA_FUNCIONARIO'}

In [ ]:
teste = "Quantos funcionarios tem no setor de marketing?"

In [ ]:
input_ids=[]
attention_masks=[]

bert_inp=tokenizer.encode_plus(teste,add_special_tokens = True,max_length =64, pad_to_max_length = True,return_attention_mask = True)
input_ids.append(bert_inp['input_ids'])
attention_masks.append(bert_inp['attention_mask'])

input_ids=np.asarray(input_ids)
attention_masks=np.array(attention_masks)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:

preds = trained_model.predict([input_ids,attention_masks],batch_size=1)


1/1 [==============================] - 0s 26ms/step


In [ ]:
teste=np.where(preds.logits>=0.50,1,0)

In [ ]:
preds.logits

array([[-1.7778617 , -0.53754526, -0.90348244, -0.8599582 ,  1.6856688 ,
        -0.44356814, -0.70820117, -1.3051971 , -0.49319133, -0.8795275 ,
         0.80512565,  5.828703  , -0.9201504 ,  2.0146732 , -0.72137386]],
      dtype=float32)

In [ ]:
for i,col in enumerate(teste[0]):
  if col == 1:
    print(dict_classes[i],preds.logits[0][i])

KeyError: 4

In [ ]:
pred_labels = np.argmax(preds.logits, axis=1)

In [ ]:
pred_labels

array([11])

In [ ]:
guia = data.label.unique()

In [ ]:
dict_classes[str(pred_labels[0])]

'METRICAS_DO_QUADRO_DE_FUNCIONARIOS'